<a href="https://colab.research.google.com/github/sreeman-11021996/Fashion-Recommendation-system/blob/main/train_resnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install import_ipynb
import import_ipynb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [3]:
!kaggle datasets download -d paramaggarwal/fashion-product-images-small

fashion-product-images-small.zip: Skipping, found more recently modified local copy (use --force to force download)


In [4]:
import zipfile
zip_ref = zipfile.ZipFile("/content/fashion-product-images-small.zip", "r")
zip_ref.extractall("/content")
zip_ref.close()

In [5]:
import tensorflow
from keras.models import Model

import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import pickle
import matplotlib.pyplot as plt

In [6]:
from utils import *
from dataset import train_val_split,FashionDataset

importing Jupyter notebook from utils.ipynb
importing Jupyter notebook from dataset.ipynb
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
#Loading the labels and removing missing images:
data=pd.read_csv('/content/styles.csv',usecols=["id","gender","masterCategory",
                                                 "subCategory","articleType",
                                                 "baseColour","season","usage"])

In [8]:
# training and validation dataset
dataset = FashionDataset(data,labels=["gender","masterCategory","subCategory"])

[INFO]: Checking if all images are present


44446it [00:03, 13576.65it/s]


[INFO]: Dropping indices: [6697, 16207, 32324, 36399, 40022]


In [11]:
dataset.df.head()

,id,classes,index
0,15970,"(Men, Apparel, Topwear)",0
1,39386,"(Men, Apparel, Bottomwear)",1
2,59263,"(Women, Accessories, Watches)",2
3,21379,"(Men, Apparel, Bottomwear)",3
4,53759,"(Men, Apparel, Topwear)",4


In [12]:
train_data, val_data = train_val_split(dataset.df)
print(f"[INFO]: Number of training sampels: {len(train_data)}")
print(f"[INFO]: Number of validation sampels: {len(val_data)}")

[INFO]: Number of training sampels: 39996
[INFO]: Number of validation sampels: 4444


In [ ]:
"""# training and validation data loader
train_dataloader = DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True
)
val_dataloader = DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False
)"""